In [ ]:
import asyncio
import threading
import time
from collections import OrderedDict, deque
from typing import TYPE_CHECKING, Dict, List
import pandas as pd
from hummingbot.client.config.config_crypt import BaseSecretsManager, ETHKeyFileSecretManger
from hummingbot.client.config.config_helpers import (
    ClientConfigAdapter,
    all_configs_complete,
    create_yml_files_legacy,
    load_client_config_map_from_file,
    load_strategy_config_map_from_file,
    read_system_configs_from_yml,
)
from hummingbot.client.config.security import Security
from hummingbot.client.hummingbot_application import HummingbotApplication
from hummingbot.connector.connector_base import ConnectorBase
from hummingbot.client.settings import STRATEGIES_CONF_DIR_PATH, AllConnectorSettings, required_exchanges
from hummingbot.client.command.gateway_command import GatewayCommand
from hummingbot.user.user_balances import UserBalances


secrets_manager_cls = ETHKeyFileSecretManger
secrets_manager = secrets_manager_cls('qoakzm,12345')
client_config_map = load_client_config_map_from_file()

if not Security.login(secrets_manager):
    print("Invalid password.")
else:
    print("login succeed.")
await Security.wait_til_decryption_done()
AllConnectorSettings.initialize_paper_trade_settings(client_config_map.paper_trade.paper_trade_exchanges)
hb = HummingbotApplication.main_application(client_config_map=client_config_map)
if not Security.is_decryption_done():
    print('  - Security check: Encrypted files are being processed. Please wait and try again later.')
else:
    print("decryption done.")

connections = await UserBalances.instance().update_exchanges(client_config_map, exchanges=required_exchanges)    
invalid_conns = {}
invalid_conns.update({ex: err_msg for ex, err_msg in connections.items() if ex in required_exchanges and err_msg is not None})
if invalid_conns:
    print('  - Exchange check: Invalid connections:')
    for ex, err_msg in invalid_conns.items():
        print(f"    {ex}: {err_msg}")
else:
    print('Exchange connection check: All connections confirmed.')

# market_status_df = pd.DataFrame(data=market.status_dict.items(), columns=["description", "status"])
# print(
#     f"  - {market.display_name.capitalize()} connector status:\n" +
#     "\n".join(["     " + line for line in market_status_df.to_string(index=False,).split("\n")])
# )    

hb.start(script="funding_rate_arb_v1.py")

login succeed.
decryption done.
Exchange connection check: All connections confirmed.
Funding feeds count: 2.
Funding feed from binance_perpetual started...
Funding feed from okx_perpetual started...


23:03:02 - script_strategy_base - binance_perpetual is not ready. Please wait...
23:03:02 - script_strategy_base - okx_perpetual is not ready. Please wait...
23:03:02 - OkxPerpetualDerivative - Network status has changed to NetworkStatus.CONNECTED. Starting networking...
23:03:02 - BinancePerpetualDerivative - Network status has changed to NetworkStatus.CONNECTED. Starting networking...
23:03:02 - BinancePerpetualUserStreamDataSource - Starting listen key management task...
23:03:02 - OkxPerpetualAPIOrderBookDataSource - Subscribed to public order book, trade and funding info channels...
23:03:02 - BinancePerpetualUserStreamDataSource - Successfully obtained listen key WecNGPDcYuJDM9cd5QJjJCrfpRngQZKoLrAWaczcFmEhHZTDxBilgFnbpa0N8uwb
23:03:02 - BinancePerpetualUserStreamDataSource - Connecting to user stream with listen key WecNGPDcYuJDM9cd5QJjJCrfpRngQZKoLrAWaczcFmEhHZTDxBilgFnbpa0N8uwb
23:03:03 - script_strategy_base - binance_perpetual is not ready. Please wait...
23:03:03 - script_s

In [ ]:
# hb.start(script="funding_rate_arb_v1.py")

# hb.start(script="log_price_example.py")
# hb.start(script="v2_funding_rate_arb_tmp.py", conf="conf_v2_funding_rate_arb_tmp_1.yml")

In [4]:
hb.stop()

23:03:52 - hummingbot_application - stop command initiated.


In [ ]:
import time
from datetime import datetime
from typing import Dict, List

_trading_pairs: List[str] = ["BTC-USDT"]
_trading_pairs_deactivation_timestamp: Dict[str, float] = {}

def initialize_trading_pairs(trading_pairs: List[str])->bool:
    initialization_success = True
    initialization_status: Dict[str, bool] = {tp: False for tp in trading_pairs}
    try:
        for tp in trading_pairs:
            # initialize...
            initialization_status[tp] = True
    except Exception as ex:
        # logging...
        pass
    
    if not all(initialization_status.values()):
        initialized_trading_pairs = [tp for tp, status in initialization_status.items() if status]
        purge_trading_pairs(initialized_trading_pairs)
        initialization_success = False
    return initialization_success
    
def purge_trading_pairs(trading_pairs: List[str]):
    try:
        for tp in trading_pairs:
            # clean up local storages and its connections to exchange...
            pass
    except Exception as ex:
        # logging...
        pass

def check_remove_trading_pairs():
    global _trading_pairs_deactivation_timestamp
    global _trading_pairs
    _tpdt = _trading_pairs_deactivation_timestamp.copy()
    for tp, ts in _trading_pairs_deactivation_timestamp.items():
        if time.time() - ts > 60:
            _trading_pairs.remove(tp)
            _tpdt.pop(tp)
    _trading_pairs_deactivation_timestamp = _tpdt
    
def activate_trading_pairs(trading_pairs: List[str])->bool:    
    global _trading_pairs_deactivation_timestamp
    global _trading_pairs
    new_trading_pairs = [tp for tp in trading_pairs if tp not in _trading_pairs]
    initialization_success = initialize_trading_pairs(new_trading_pairs)
    if initialization_success:
        _trading_pairs.extend(new_trading_pairs)
        for tp in trading_pairs:
            if tp in _trading_pairs_deactivation_timestamp:
                _trading_pairs_deactivation_timestamp.pop(tp)
    check_remove_trading_pairs()
    return initialization_success

def deactivate_trading_pairs(trading_pairs: List[str]):
    global _trading_pairs_deactivation_timestamp
    global _trading_pairs
    purge_trading_pairs(trading_pairs)
    for tp in trading_pairs:
        if tp in _trading_pairs and tp not in _trading_pairs_deactivation_timestamp:
            _trading_pairs_deactivation_timestamp[tp] = time.time()
    check_remove_trading_pairs()

def status_print():
    global _trading_pairs_deactivation_timestamp
    global _trading_pairs
    info = f"trading_pairs: {_trading_pairs} \n" 
    info += f"_trading_pairs_deactivation_timestamp: "
    for tp, ts in _trading_pairs_deactivation_timestamp.items():
        info += f"{tp} {datetime.fromtimestamp(ts).strftime("%y-%m-%d %H:%M:%S")} "
    print(info)

In [ ]:
status_print()

activate_trading_pairs(["ETH-USDT", "SOL-USDT"])
deactivate_trading_pairs(["BTC-USDT"])
status_print()

activate_trading_pairs(["ETH-USDT"])
deactivate_trading_pairs(["SOL-USDT"])
status_print()

activate_trading_pairs(["SOL-USDT"])
deactivate_trading_pairs(["ETH-USDT"])
status_print()

activate_trading_pairs(["ETH-USDT", "SOL-USDT"])
deactivate_trading_pairs(["BTC-USDT"])
status_print()

activate_trading_pairs(["ETH-USDT"])
deactivate_trading_pairs(["BTC-USDT", "SOL-USDT"])
status_print()

activate_trading_pairs(["BTC-USDT"])
deactivate_trading_pairs(["ETH-USDT"])
status_print()

activate_trading_pairs(["BTC-USDT"])
status_print()

In [1]:
import time
import copy
from datetime import datetime
from decimal import Decimal
import requests
from typing import Dict, List
from pathlib import Path

# now = datetime.now()
# now.strftime("%H:%M:%S.%f")
# now.strftime("%H:%M:%S")

# pre = print(datetime.now())
# server_time = datetime.fromtimestamp(requests.get("https://fapi.binance.com/fapi/v1/time").json()["serverTime"] * 1e-3)
# print(server_time)
# post = print(datetime.now())

# print(round(Decimal(0.33321321),6))

# l=[None,None]
# s=[i for i in l if i]
# # print(s)
# print(max(s))

# print(time.time()) #1970-01-01 00:00:00距今的秒数
# print(time.time()/60/60/24/365)
# print((600.0-time.time()%600.0)/60.0)
# print((1754064000000-1754049600000)/1000/60/60)

# datetime.fromtimestamp(time.time()).strftime("%y-%m-%d %H:%M:%S")

# i = 0
# while True:
#     i += 1
#     if i > 20:
#         break
#     if time.time() % 5.0 < 1.0:
#         print("10s reached!")
#     time.sleep(1) 

# p = Path(".")
# p.absolute()

# class SomeClass:
#     def __init__(self, id = []):
#         self._id_list = id
        
#     @property
#     def id_list(self):
#         return self._id_list
    
#     @id_list.setter
#     def id_list(self, id):
#         self._id_list = id
        
# sc = SomeClass([1, 2, 3])
# print(sc.id_list)
# il = sc.id_list
# il.append(4)
# print(sc.id_list)
# sc.id_list.append(5)
# print(sc.id_list)
# print(il)
# sc.id_list = il
# print(sc.id_list)

# il1 = list(copy.deepcopy(sc.id_list))
# il1.append(6)
# print(il1)
# print(sc.id_list)
# sc.id_list = il1
# print(sc.id_list)

# print(all([]))

l=[1,2,3,2]
l.remove(2)
print(l)


[1, 3, 2]


In [2]:

from hummingbot import root_path
from pathlib import Path
from hummingbot.client.settings import SCRIPT_STRATEGIES_MODULE, SCRIPT_STRATEGIES_PATH, STRATEGIES
from hummingbot.client.config.config_helpers import load_client_config_map_from_file

print(root_path())
client_config_map = load_client_config_map_from_file()
print(Path(SCRIPT_STRATEGIES_PATH))

/home/wangbin/repo/hbext
/home/wangbin/repo/hbext/scripts


In [ ]:
import requests
import ccxt
 
api = ccxt.binance({
    "enableRateLimit": True,
    "proxies": {
        "http": "127.0.0.1:7897",
        "https": "127.0.0.1:7897",
    },
    'apiKey':'**',
    'secret':'**',
})

# public api
markets = api.load_markets()  # 获取Binance所有交易对信息
print(len(markets))
ticker = api.fetch_ticker('BTC/USDT')  # 获取 BTC/USDT 的最新市场数据
print(ticker)
exchange_name = api.name
print(exchange_name)

# private api
balance = api.fetch_balance(params={"omitZeroBalances":"true"})
print(balance)  
